# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [127]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
from cartopy import crs

# Import API key
from api_keys import geoapify_key

In [128]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,7.14,84,54,5.28,GS,1706589132
1,1,riachao das neves,-11.7461,-44.9100,20.55,92,28,0.51,BR,1706589133
2,2,longyearbyen,78.2186,15.6401,-2.09,74,75,5.14,SJ,1706589133
3,3,flying fish cove,-10.4217,105.6791,26.82,94,100,2.06,CX,1706589133
4,4,puerto natales,-51.7236,-72.4875,8.94,95,100,0.97,CL,1706589133


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [129]:
%%capture --no-display


# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color="City",
    scale=2
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [130]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_n = city_data_df[(city_data_df["Max Temp"] < 32) & 
                            (city_data_df["Max Temp"] > 12) &
                            (city_data_df["Humidity"] < 50) &
                            (city_data_df["Wind Speed"] < 10) &
                            (city_data_df["Cloudiness"] < 15)]

# Drop any rows with null values
city_data_df_n = city_data_df_n.dropna()

# Display sample data
city_data_df_n.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,kidal,18.4411,1.4078,15.59,21,0,5.92,ML,1706589136
81,81,mount gambier,-37.8333,140.7667,21.67,38,0,5.21,AU,1706589150
88,88,zouerate,22.7187,-12.4521,15.21,17,0,3.76,MR,1706589151
174,174,el jadid,33.2549,-8.5060,15.62,42,1,3.10,MA,1706589168
194,194,zalingei,12.9096,23.4741,16.11,25,9,2.51,SD,1706589172


### Step 3: Create a new DataFrame called `hotel_df`.

In [131]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_n[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
16,kidal,ML,18.4411,1.4078,21,
81,mount gambier,AU,-37.8333,140.7667,38,
88,zouerate,MR,22.7187,-12.4521,17,
174,el jadid,MA,33.2549,-8.5060,42,
194,zalingei,SD,12.9096,23.4741,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [132]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kidal - nearest hotel: No hotel found
mount gambier - nearest hotel: The Old Mount Gambier Gaol
zouerate - nearest hotel: فندق تيرس زمور
el jadid - nearest hotel: Hotel El Jadida فندق الجديدة
zalingei - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
los frentones - nearest hotel: No hotel found
longquan - nearest hotel: 仁和宾馆
xichang - nearest hotel: No hotel found
chacabuco - nearest hotel: Hotel Postal
gao - nearest hotel: No hotel found
pathein - nearest hotel: Taan Taan Ta
muang phonsavan - nearest hotel: COECCO Xieng Khouang Hotel
thayetmyo - nearest hotel: No hotel found
capayan - nearest hotel: Hosteria Concepción
upington - nearest hotel: Die Eiland Holiday Ressort
sidi ifni - nearest hotel: Hotel Ére Nouvelle
nouadhibou - nearest hotel: Hotel valencia
dadukou - nearest hotel: 锦江之星酒店
rioja - nearest hotel: Hotel Avenida


,City,Country,Lat,Lng,Humidity,Hotel Name
16,kidal,ML,18.4411,1.4078,21,No hotel found
81,mount gambier,AU,-37.8333,140.7667,38,The Old Mount Gambier Gaol
88,zouerate,MR,22.7187,-12.4521,17,فندق تيرس زمور
174,el jadid,MA,33.2549,-8.5060,42,Hotel El Jadida فندق الجديدة
194,zalingei,SD,12.9096,23.4741,25,No hotel found
204,tura,IN,25.5198,90.2201,29,No hotel found
240,los frentones,AR,-26.4077,-61.4137,45,No hotel found
291,longquan,CN,24.6719,102.1613,48,仁和宾馆
320,xichang,CN,27.8964,102.2634,43,No hotel found
337,chacabuco,AR,-34.6417,-60.4739,46,Hotel Postal


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [133]:
%%capture --no-display


# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols = ["Hotel Name", "Country"],
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    scale = 3
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)